# Data science tips
## 시계열을 지도학습 문제로 변환하기

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div> 

<div style="text-align: right"> Initial upload: 2020.11.08 </div> 
<div style="text-align: right"> Last update: 2020.11.08</div> 

- 참고자료 : https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

시계열 예측을 위해 머신러닝이 사용될 수 있다.  이때 시계열 데이터셋이 지도학습에 적합하도록 reframe 되어야 한다.  
여기서는 시계열 데이터를 지도학습용 데이터셋으로 변형하는 방법에 대하여 살펴본다.

### 1. 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import datetime
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline


예를들어 다음과 같은 시계열이 있다고 가정하자.  
0, 1, 2, 3, 4, 5...  

그리고 입력으로 다은 스텝의 값을 예측하고자 한다면 데이터 셋은 다음과 같이 바껴야 할 것이다.  
X: 0, 1, 2, 3, 4 ..  
y : 1, 2, 3, 4, 5 ..

이때 pandas shift함수를 사용하면 된다.  
예를 들면

In [2]:
df = pd.DataFrame()
df['t'] = [ x for x in range(10)]

In [3]:
df

,t
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [4]:
df['t-1'] = df['t'].shift(1)

In [5]:
df

,t,t-1
0,0,NaN
1,1,0.0
2,2,1.0
3,3,2.0
4,4,3.0
5,5,4.0
6,6,5.0
7,7,6.0
8,8,7.0
9,9,8.0


In [6]:
df['t+1'] = df['t'].shift(-1)

In [7]:
df

,t,t-1,t+1
0,0,NaN,1.0
1,1,0.0,2.0
2,2,1.0,3.0
3,3,2.0,4.0
4,4,3.0,5.0
5,5,4.0,6.0
6,6,5.0,7.0
7,7,6.0,8.0
8,8,7.0,9.0
9,9,8.0,NaN


이방법 을 사용하여 Series_to_supervised() 함수를 만든다.

In [20]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: list 또는 시계열로된 데이터.
        n_in: 입력 x가 될 lag 수.
        n_out: 출력 y가 될 lead 수.
        dropnan: Null 삭제여부.
    Returns:
        reframe된 데이터프레임.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        if i-1 == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t-%d)' % (j+1, i-1)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        names += [('var%d(t+%d)' % (j+1, i+1)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### 2. single input - single output : one step forecasting
위 함수를 사용하여 1 step univarate forecasting용 데이터셋을 만들어 본다.

In [21]:
values = [x for x in range(10)]
data = series_to_supervised(values)

In [22]:
data

,var1(t),var1(t+1)
1,0.0,1
2,1.0,2
3,2.0,3
4,3.0,4
5,4.0,5
6,5.0,6
7,6.0,7
8,7.0,8
9,8.0,9


3개의 lag를 입력으로 하고 싶다면  

In [23]:
data = series_to_supervised(values, 3)

In [24]:
data

,var1(t-2),var1(t-1),var1(t),var1(t+1)
3,0.0,1.0,2.0,3
4,1.0,2.0,3.0,4
5,2.0,3.0,4.0,5
6,3.0,4.0,5.0,6
7,4.0,5.0,6.0,7
8,5.0,6.0,7.0,8
9,6.0,7.0,8.0,9


### 3. single input - single output : multi step forecasting

In [25]:
values = [x for x in range(10)]
data = series_to_supervised(values, 2, 2)

In [26]:
data

,var1(t-1),var1(t),var1(t+1),var1(t+2)
2,0.0,1.0,2,3.0
3,1.0,2.0,3,4.0
4,2.0,3.0,4,5.0
5,3.0,4.0,5,6.0
6,4.0,5.0,6,7.0
7,5.0,6.0,7,8.0
8,6.0,7.0,8,9.0


### 4. multi input - single output : one step forecasting
입력이 여러개인 multivariate time series의 경우에도 유사하다.

In [27]:
raw = pd.DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]

In [28]:
values = raw.values
data = series_to_supervised(values)

In [29]:
data

,var1(t),var2(t),var1(t+1),var2(t+1)
1,0.0,50.0,1,51
2,1.0,51.0,2,52
3,2.0,52.0,3,53
4,3.0,53.0,4,54
5,4.0,54.0,5,55
6,5.0,55.0,6,56
7,6.0,56.0,7,57
8,7.0,57.0,8,58
9,8.0,58.0,9,59


### 5. multi input - single output : multi step forecasting
이제 multi step 데이터셋을 만들어보자

In [30]:
raw = pd.DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values, 1, 2)

In [31]:
data

,var1(t),var2(t),var1(t+1),var2(t+1),var1(t+2),var2(t+2)
1,0.0,50.0,1,51,2.0,52.0
2,1.0,51.0,2,52,3.0,53.0
3,2.0,52.0,3,53,4.0,54.0
4,3.0,53.0,4,54,5.0,55.0
5,4.0,54.0,5,55,6.0,56.0
6,5.0,55.0,6,56,7.0,57.0
7,6.0,56.0,7,57,8.0,58.0
8,7.0,57.0,8,58,9.0,59.0
